In [17]:
# Import packages
 
# Fake data manipulation
from faker import Faker
import random
import itertools
# Import functions from own files
from sql_fun import *
from alter_check_db_fun import * 
from constants import *
from bot_fun import *
from queries_fun import *


### Generate tables relationship included (employee, service, working_days, emp_s )

In [18]:
# Delete all tables and create them again
try:
    del_all_tables(path)
except OperationalError:
    print("Such database doesn't exist")

 The following table was deleted: employee
 The following table was deleted: service
 The following table was deleted: emp_service
 The following table was deleted: working_days
 The following table was deleted: booked_appointments
 The following table was deleted: working_hours


In [19]:
emp_cols ={"first_name" : "TEXT",
 "last_name" : "TEXT", 
 "email" : "TEXT",
 "current_status" : "TEXT"}

e = gen_tabhead_query("employee", emp_cols, primary= True)
###
serv_cols = {'service_name' : "TEXT", 
'service_price_eu' : "INTEGER"}

s = gen_tabhead_query("service", serv_cols, primary= True)

###
emp_serv_cols = {'employee_id': "INTEGER", 
                'service_id' : 'INTEGER'}

f_keys_emp_s = ['service_id', 'employee_id']
referto_emp_s = {'service' : 'service_id', "employee":"employee_id"}
es = gen_tabhead_query("emp_service", emp_serv_cols, primary=True, foreign = f_keys_emp_s, reference_tab_col=referto_emp_s)

# By adding a primary key to emp_service table we can refer to it in orders table later on 


###
d_cols = {"employee_id" : "INTEGER", 
 "day_of_the_week" : "TEXT"}

foreign_d = ['employee_id']
referto_days = {"employee" : "employee_id"}

d = gen_tabhead_query("working_days", d_cols, primary = True, foreign = foreign_d, reference_tab_col= referto_days)

###
###

orders_cols = {"emp_service_id" : "INTEGER", 
"user_name" : "TEXT",  
"telegram_id" : "INTEGER",
"execution_time" : "TEXT", 
"execution_date" : "TEXT",
"status": "TEXT" }

f_keys = ["emp_service_id"]
ref = {"emp_service" : "emp_service_id"}

b = gen_tabhead_query("booked_appointments", orders_cols, primary = True, foreign = f_keys, reference_tab_col=ref )

###
###

h_cols = {"wd_emp" : "INTEGER",
"start_time" : "TEXT"}

wd_emp_f = ["wd_emp"]
referto_h = {'working_days': "working_days_id"}
h = gen_tabhead_query("working_hours", h_cols, primary = False, foreign = wd_emp_f, reference_tab_col=referto_h )

###
###
print()
print(e)
print(s)
print(es)
print(d)
print(b)
print(h)




CREATE TABLE employee (employee_id INTEGER PRIMARY KEY, first_name TEXT, last_name TEXT, email TEXT, current_status TEXT)
CREATE TABLE service (service_id INTEGER PRIMARY KEY, service_name TEXT, service_price_eu INTEGER)
CREATE TABLE emp_service (emp_service_id INTEGER PRIMARY KEY, employee_id INTEGER, service_id INTEGER, FOREIGN KEY (service_id) REFERENCES  service(service_id), FOREIGN KEY (employee_id) REFERENCES  employee(employee_id))
CREATE TABLE working_days (working_days_id INTEGER PRIMARY KEY, employee_id INTEGER, day_of_the_week TEXT, FOREIGN KEY (employee_id) REFERENCES  employee(employee_id))
CREATE TABLE booked_appointments (booked_appointments_id INTEGER PRIMARY KEY, emp_service_id INTEGER, user_name TEXT, telegram_id INTEGER, execution_time TEXT, execution_date TEXT, status TEXT, FOREIGN KEY (emp_service_id) REFERENCES  emp_service(emp_service_id))
CREATE TABLE working_hours (wd_emp INTEGER, start_time TEXT, FOREIGN KEY (wd_emp) REFERENCES  working_days(working_days_id))

In [20]:
# Connect to the database and create tables 
conn =sql.connect(path)
c = conn.cursor()
###

# Create employee table
c.execute(e)
# Create service table
c.execute(s)
# Create table which links services and employees they provide
c.execute(es)
# Create working days table
c.execute(d)
# Table with booked appointments
c.execute(b)
# Table with working hours
c.execute(h)



conn.commit()
conn.close()

### Add data to employees table

In [21]:
fake = Faker()
Faker.seed(678)
for i in range(no_workers):
    # Generate fake data
    row = (fake.first_name(), fake.last_name(), fake.email(), "ACTIVE")
    add_row(path, "employee", row)

show_table_all(path, "employee")


,employee_id,first_name,last_name,email,current_status
0,1,Daniel,Flores,lori27@example.net,ACTIVE
1,2,Jennifer,Rojas,andersonconnor@example.com,ACTIVE
2,3,Benjamin,Cruz,tinabradley@example.com,ACTIVE
3,4,Lori,Stein,laurajohnson@example.net,ACTIVE


#### Add data to services table

In [22]:
random.seed(10)

for i in range(number_of_services):
    name = f"service_{i+1}"
    p = random.randint(15, 60)
    add = (name, p)
    add_row(path, "service", add)

show_table_all(path, "service")

,service_id,service_name,service_price_eu
0,1,service_1,51
1,2,service_2,17
2,3,service_3,42
3,4,service_4,45
4,5,service_5,51
5,6,service_6,15
6,7,service_7,28
7,8,service_8,44


### Add data to emp_service table 

In [23]:
# Get emp id as a list

# Create conn
conn = sql.connect(path)
# Cursor
c = conn.cursor()
# sql command
query = "SELECT employee_id FROM employee"
# execute
c.execute(query)
# save results
data = c.fetchall()

# Close connection
conn.close()
emp_id = [i[0] for i in data]
emp_id

[1, 2, 3, 4]

In [24]:
# Get service id as a list

# Create conn
conn = sql.connect(path)
# Cursor
c = conn.cursor()
# sql command
query = "SELECT service_id FROM service"
# execute
c.execute(query)
# save results
data = c.fetchall()
# Close connection
conn.close()
serv_id = [i[0] for i in data]
serv_id

[1, 2, 3, 4, 5, 6, 7, 8]

In [25]:
random.seed(5)
possible_associations = [ca for ca in itertools.product(emp_id, serv_id)]
df = (
    pd.DataFrame.from_records(
        random.choices(possible_associations, k=20),
        columns=["employee_id", "service_id"],
    )
    .sort_values(["employee_id", "service_id"])
    .reset_index()
    .drop(columns="index")
    .drop_duplicates()
)

print(df)
print("Unique employee ids:", np.sort(df.employee_id.unique()))
print("Unique service ids:", np.sort(df.service_id.unique()))

    employee_id  service_id
0             1           1
2             1           4
3             1           7
4             1           8
5             2           1
6             2           7
7             2           8
8             3           2
9             3           3
10            3           4
11            3           5
12            3           8
14            4           2
15            4           5
16            4           6
18            4           7
Unique employee ids: [1 2 3 4]
Unique service ids: [1 2 3 4 5 6 7 8]


In [26]:
conn = sql.connect(path)
df.to_sql("emp_service", conn, if_exists = "append", index = False)
conn.commit()
conn.close()

show_table_all(path, "emp_service")

,emp_service_id,employee_id,service_id
0,1,1,1
1,2,1,4
2,3,1,7
3,4,1,8
4,5,2,1
5,6,2,7
6,7,2,8
7,8,3,2
8,9,3,3
9,10,3,4


In [27]:
# Join the employee table and service table based on the data from emp_service table

# Connect to the database
conn = sql.connect(path)
# Write sql command
query = """SELECT  es.emp_service_id, es.employee_id, es.service_id, e.first_name, e.last_name, s.service_name
            FROM emp_service es
            LEFT JOIN employee e  
            ON es.employee_id = e.employee_id
            LEFT JOIN service s
            ON es.service_id = s.service_id """
pd.read_sql_query(
    query,
conn)

,emp_service_id,employee_id,service_id,first_name,last_name,service_name
0,1,1,1,Daniel,Flores,service_1
1,2,1,4,Daniel,Flores,service_4
2,3,1,7,Daniel,Flores,service_7
3,4,1,8,Daniel,Flores,service_8
4,5,2,1,Jennifer,Rojas,service_1
5,6,2,7,Jennifer,Rojas,service_7
6,7,2,8,Jennifer,Rojas,service_8
7,8,3,2,Benjamin,Cruz,service_2
8,9,3,3,Benjamin,Cruz,service_3
9,10,3,4,Benjamin,Cruz,service_4


### Add data to the table with working days

In [28]:
add_days(path, 1, "monday", "wednesday", 'friday')
add_days(path, 2, "monday", "tuesday")
add_days(path, 3, "wednesday", "thursday", "friday")
add_days(path, 4, "tuesday", "thursday")

In [29]:
show_table_all(path, "working_days")

,working_days_id,employee_id,day_of_the_week
0,1,1,Monday
1,2,1,Wednesday
2,3,1,Friday
3,4,2,Monday
4,5,2,Tuesday
5,6,3,Wednesday
6,7,3,Thursday
7,8,3,Friday
8,9,4,Tuesday
9,10,4,Thursday


### Add data to the table with working hours

In [30]:
# Join the employee table and service table based on the data from emp_service table
query = """ SELECT w.working_days_id, e.employee_id, w.day_of_the_week, e.first_name, e.last_name
            FROM working_days w
            JOIN employee e 
            ON w.employee_id = e.employee_id
            WHERE w.day_of_the_week = 'Friday'"""

show_table_from_q(path, query)

,working_days_id,employee_id,day_of_the_week,first_name,last_name
0,3,1,Friday,Daniel,Flores
1,8,3,Friday,Benjamin,Cruz


In [31]:
# Monday
add_hours(path, 1, "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00")
add_hours(path, 4, "09:00", "10:00", "11:00", "16:00", "17:00", "18:00", "19:00")

# Tuesday
add_hours(path, 5, "14:00", "15:00", "16:00", "17:00", "18:00", "19:00")
add_hours(path, 9, "09:00", "10:00", "11:00", "12:00", "13:00")

# Wednesday
add_hours(path, 2, "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00")
add_hours(path, 6, "09:00", "10:00", "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00")

# Thursday 
add_hours(path, 7, "09:00", "10:00", "11:00", "12:00", "13:00")
add_hours(path, 10, "16:00", "17:00", "18:00", "19:00")

# Friday 
add_hours(path, 3, "09:00", "10:00", "11:00", "12:00", "13:00", "15:00", "16:00", "17:00", "18:00", "19:00")
add_hours(path, 8, "09:00", "10:00", "11:00", "12:00", "13:00", "15:00", "16:00", "17:00", "18:00", "19:00")


	(1, 09:00) were added to working_hours table;

	(1, 10:00) were added to working_hours table;

	(1, 11:00) were added to working_hours table;

	(1, 12:00) were added to working_hours table;

	(1, 13:00) were added to working_hours table;

	(1, 14:00) were added to working_hours table;

	(1, 15:00) were added to working_hours table;

	(1, 16:00) were added to working_hours table;

	(1, 17:00) were added to working_hours table;

	(1, 18:00) were added to working_hours table;

	(1, 19:00) were added to working_hours table;

	(4, 09:00) were added to working_hours table;

	(4, 10:00) were added to working_hours table;

	(4, 11:00) were added to working_hours table;

	(4, 16:00) were added to working_hours table;

	(4, 17:00) were added to working_hours table;

	(4, 18:00) were added to working_hours table;

	(4, 19:00) were added to working_hours table;

	(5, 14:00) were added to working_hours table;

	(5, 15:00) were added to working_hours table;

	(5, 16:00) were added to working_hours

In [32]:
show_table_all(path, "working_hours")

,wd_emp,start_time
0,1,09:00
1,1,10:00
2,1,11:00
3,1,12:00
4,1,13:00
...,...,...
75,8,15:00
76,8,16:00
77,8,17:00
78,8,18:00
